In [1]:
##98.6%
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import helpermethods
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2)
import numpy as np
import sys
from tensorflow.keras.layers import Dense, Input, RNN
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

#FastRNN and FastGRNN imports
from rnn import FastGRNNCellKeras, FastRNNCellKeras
from data_utils import *

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6



/home/nesl/209as_sec/human_act/TinyML Models/fastgrnn keras/rnn.py:1826: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert len(self.varList) is 0
/home/nesl/209as_sec/human_act/TinyML Models/fastgrnn keras/rnn.py:1948: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert len(self.varList) is 0
/home/nesl/209as_sec/human_act/TinyML Models/fastgrnn keras/rnn.py:2086: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert len(self.varList) is 0
/home/nesl/209as_sec/human_act/TinyML Models/fastgrnn keras/rnn.py:2281: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert len(self.varList) is 0
/home/nesl/209as_sec/human_act/TinyML Models/fastgrnn keras/rnn.py:2417: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert len(self.varList) is 0


In [2]:
f = '/home/nesl/209as_sec/human_act/Data/Activity_Dataset/'
model_dir = 'trained_models/'
window_size = 550
stride = 50

X_tr, Y_tr, X_test, Y_test = import_auritus_activity_dataset(dataset_folder = f, 
                                use_timestamp=False, 
                                shuffle=True, 
                                window_size = window_size, stride = stride, 
                                return_test_set = True, test_set_size = 300)
print(X_tr.shape)
print(Y_tr.shape)
print(X_test.shape)
print(Y_test.shape)

Xtrain = X_tr
Ytrain = Y_tr
Xtest = X_test
Ytest = Y_test
numClasses = Y_tr.shape[1]
dataDimension = Xtrain[0].shape
channels = Xtrain.shape[2]

100%|█████████████████████████████████████████| 399/399 [00:12<00:00, 30.90it/s]


(14671, 550, 6)
(14671, 9)
(300, 550, 6)
(300, 9)


In [3]:
cell = "FastGRNN" # Choose between FastGRNN, FastRNN  ###, UGRNN, GRU and LSTM

inputDims = [window_size, channels] #features taken in by RNN in one timestep
hiddenDims = 50 #hidden state of RNN

totalEpochs = 300
batchSize = 100

learningRate = 0.01
decayStep = 200
decayRate = 0.1

outFile = None #provide your file, if you need all the logging info in a file

#low-rank parameterisation for weight matrices. None => Full Rank
wRank = None 
uRank = None 

#Sparsity of the weight matrices. x => 100*x % are non-zeros
#Note: Sparsity inducing is not supported in this code. 
sW = 1.0 
sU = 1.0

#Non-linearities for the RNN architecture. Can choose from "tanh, sigmoid, relu, quantTanh, quantSigm"
update_non_linearity = "tanh"
gate_non_linearity = "sigmoid"


FastCell = FastGRNNCellKeras(hiddenDims, gate_non_linearity=gate_non_linearity,
                        update_non_linearity=update_non_linearity,
                        wRank=wRank, uRank=uRank)

In [4]:
x = inputs = Input(shape=inputDims, name='input')
layer_shape = K.int_shape(x)#x.get_shape()
print(layer_shape)
x = RNN(FastCell, return_sequences=False, name='rnn')(x)
out = Dense(numClasses, activation='softmax', name='dense')(x)
model = Model(inputs=inputs, outputs=out)
Adam_ = Adam()
model.compile(optimizer=Adam_, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

(None, 550, 6)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 550, 6)]          0         
_________________________________________________________________
rnn (RNN)                    (None, 50)                2902      
_________________________________________________________________
dense (Dense)                (None, 9)                 459       
Total params: 3,361
Trainable params: 3,361
Non-trainable params: 0
_________________________________________________________________


/home/nesl/swapnil_debug/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:1331: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


In [5]:
model_name = 'fastgrnn.h5'
checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(Xtrain, Ytrain, batchSize, epochs=totalEpochs, validation_data=(Xtest, Ytest),callbacks=[checkpoint])

Epoch 1/300
147/147 [==============================] - 84s 561ms/step - loss: 1.8119 - accuracy: 0.3391 - val_loss: 1.4077 - val_accuracy: 0.4633
Epoch 2/300
147/147 [==============================] - 84s 573ms/step - loss: 1.2602 - accuracy: 0.4876 - val_loss: 1.1384 - val_accuracy: 0.5133
Epoch 3/300
147/147 [==============================] - 85s 576ms/step - loss: 1.1050 - accuracy: 0.5400 - val_loss: 1.0280 - val_accuracy: 0.5300
Epoch 4/300
147/147 [==============================] - 85s 579ms/step - loss: 0.9918 - accuracy: 0.5810 - val_loss: 0.9199 - val_accuracy: 0.6033
Epoch 5/300
147/147 [==============================] - 85s 578ms/step - loss: 0.8503 - accuracy: 0.6474 - val_loss: 0.7699 - val_accuracy: 0.6967
Epoch 6/300
147/147 [==============================] - 85s 579ms/step - loss: 0.7559 - accuracy: 0.6939 - val_loss: 0.6474 - val_accuracy: 0.7533
Epoch 7/300
147/147 [==============================] - 85s 576ms/step - loss: 0.6853 - accuracy: 0.7254 - val_loss: 0.5998 -

Epoch 57/300
147/147 [==============================] - 86s 586ms/step - loss: 0.1927 - accuracy: 0.9205 - val_loss: 0.1975 - val_accuracy: 0.9233
Epoch 58/300
147/147 [==============================] - 85s 578ms/step - loss: 0.2130 - accuracy: 0.9145 - val_loss: 0.1892 - val_accuracy: 0.9233
Epoch 59/300
147/147 [==============================] - 85s 579ms/step - loss: 0.1947 - accuracy: 0.9189 - val_loss: 0.2229 - val_accuracy: 0.9233
Epoch 60/300
147/147 [==============================] - 85s 577ms/step - loss: 0.2147 - accuracy: 0.9145 - val_loss: 0.1920 - val_accuracy: 0.9300
Epoch 61/300
147/147 [==============================] - 85s 580ms/step - loss: 0.1899 - accuracy: 0.9228 - val_loss: 0.1796 - val_accuracy: 0.9367
Epoch 62/300
147/147 [==============================] - 85s 581ms/step - loss: 0.1862 - accuracy: 0.9217 - val_loss: 0.1944 - val_accuracy: 0.9267
Epoch 63/300
147/147 [==============================] - 85s 576ms/step - loss: 0.1754 - accuracy: 0.9277 - val_loss: 0

147/147 [==============================] - 85s 581ms/step - loss: 0.0877 - accuracy: 0.9672 - val_loss: 0.1186 - val_accuracy: 0.9533
Epoch 168/300
147/147 [==============================] - 85s 578ms/step - loss: 0.0941 - accuracy: 0.9653 - val_loss: 0.1111 - val_accuracy: 0.9633
Epoch 169/300
147/147 [==============================] - 85s 578ms/step - loss: 0.1367 - accuracy: 0.9502 - val_loss: 0.1464 - val_accuracy: 0.9500
Epoch 170/300
147/147 [==============================] - 85s 581ms/step - loss: 0.0799 - accuracy: 0.9728 - val_loss: 0.0843 - val_accuracy: 0.9800
Epoch 171/300
147/147 [==============================] - 84s 574ms/step - loss: 0.0798 - accuracy: 0.9716 - val_loss: 0.1127 - val_accuracy: 0.9600
Epoch 172/300
147/147 [==============================] - 85s 577ms/step - loss: 0.0848 - accuracy: 0.9695 - val_loss: 0.1237 - val_accuracy: 0.9433
Epoch 173/300
147/147 [==============================] - 85s 576ms/step - loss: 0.1062 - accuracy: 0.9641 - val_loss: 0.1518 -

147/147 [==============================] - 87s 595ms/step - loss: 0.0974 - accuracy: 0.9678 - val_loss: 0.1366 - val_accuracy: 0.9633
Epoch 278/300
147/147 [==============================] - 87s 594ms/step - loss: 0.0433 - accuracy: 0.9866 - val_loss: 0.0542 - val_accuracy: 0.9833
Epoch 279/300
147/147 [==============================] - 88s 601ms/step - loss: 0.0399 - accuracy: 0.9879 - val_loss: 0.0516 - val_accuracy: 0.9867
Epoch 280/300
147/147 [==============================] - 88s 600ms/step - loss: 0.0502 - accuracy: 0.9839 - val_loss: 0.0714 - val_accuracy: 0.9667
Epoch 281/300
147/147 [==============================] - 88s 600ms/step - loss: 0.0461 - accuracy: 0.9851 - val_loss: 0.0627 - val_accuracy: 0.9767
Epoch 282/300
147/147 [==============================] - 87s 593ms/step - loss: 0.0552 - accuracy: 0.9825 - val_loss: 0.0991 - val_accuracy: 0.9700
Epoch 283/300
147/147 [==============================] - 88s 596ms/step - loss: 0.0991 - accuracy: 0.9660 - val_loss: 0.0947 -

KeyboardInterrupt: 

In [6]:
model = load_model(model_name, custom_objects={'FastGRNNCellKeras':FastCell})
model.evaluate(Xtest,Ytest)

10/10 [==============================] - 1s 68ms/step - loss: 0.0619 - accuracy: 0.9867


[0.061922404915094376, 0.9866666793823242]